In [5]:
import os
import pickle
import timeit
from datetime import datetime

In [6]:
nlct_dir_path = "/home/jturner/false_low_clouds_2023/Nighttime_Low_Cloud_Test/nlct_data/"

In [17]:
nlct_dates = sorted(os.listdir(nlct_dir_path))

#---Removing any non-date directory items
nlct_dates = [x for x in nlct_dates if len(x) == 8]

nlct_dates

['20220219', '20220914', '20230417', '20230427', '20230901', '20230907']

In [ ]:
def find_static_features(BTD_c, nlct_dir_path, nlct_date):

    #--- Initialize an array of 1's in the shape of the final image
    BTD_prod = [ [1]*len(BTD_c[0][0]) for i in range(len(BTD_c[0]))]

    #--- Starting the timer
    starttime = timeit.default_timer()

    for filename in sorted(os.listdir(nlct_path)):
            print(filename)

            #--- Open the NLCT file
            file = open(nlct_dir_path+nlct_date+'/'+filename, 'rb')
            BTD = pickle.load(file)
            file.close()

            #--- Crop to the specified region
            BTD = BTD.where((BTD.lat>min_lat) & (BTD.lat<max_lat) & (BTD.lon>min_lon) & (BTD.lon<max_lon))
            BTD = BTD.dropna(dim="y", how="all")
            BTD = BTD.dropna(dim="x", how="all")

            #--- Set negative values to zero
            BTD = np.where(BTD < 0, 0, BTD)

            #--- Multiply to only enhance static clouds
            BTD_prod = BTD_prod*BTD

            #--- Limit runaway large values to 3
            BTD_prod = np.where(BTD_prod > 3, 3, BTD_prod)
            
            #--- Convert BTD prod (2d array) to value
            static_value = np.nansum(BTD_prod)
            
            #--- Create date variable
            date = datetime.strptime(nlct_date, "%Y%m%d")

    print("Completed ", date, ": ", timeit.default_timer() - starttime)
    
    return date, static_value
    